# Neural Network Example

Build a 2-hidden layers fully connected neural network (a.k.a multilayer perceptron) with TensorFlow.

This example is using some of TensorFlow higher-level wrappers (tf.estimators, tf.layers, tf.metrics, ...), you can check 'neural_network_raw' example for a raw, and more detailed TensorFlow implementation.

- Author: Aymeric Damien
- Project: https://github.com/aymericdamien/TensorFlow-Examples/

## Neural Network Overview

<img src="http://cs231n.github.io/assets/nn1/neural_net2.jpeg" alt="nn" style="width: 400px;"/>

## MNIST Dataset Overview

This example is using MNIST handwritten digits. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1. For simplicity, each image has been flattened and converted to a 1-D numpy array of 784 features (28*28).

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

More info: http://yann.lecun.com/exdb/mnist/

In [2]:
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

/Users/lvst/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.1
num_steps = 1000
batch_size = 32
display_step = 100

# Network Parameters
n_hidden_1 = 32 # 1st layer number of neurons
n_hidden_2 = 32 # 2nd layer number of neurons
num_input = 32 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

In [6]:
import pandas as pd

datapath = './'

train_df = pd.read_csv(datapath+'[new] yancheng_train_20171226.csv')

test_df = pd.read_csv(datapath+'yancheng_testA_20171225.csv')

print(train_df.shape)

train_sum10=train_df[(train_df.sale_date==201710)].groupby(['class_id']).sale_quantity.sum().round()
group_by_class=train_df.groupby(['class_id'])

print(len(group_by_class.groups))

groups=group_by_class.groups

print(type(groups))

for key, value in groups.items():
    print(key)
    print(group_by_class.get_group(key).sort_values(by=['sale_date','TR','gearbox_type','engine_torque']).shape)
    
    
#     print(type(value))
#     print(group_by_class.get_group(key))

# print(type(group_by_class.get_group(103507)))
# print(group_by_class.get_group(103507).shape)    
# print(group_by_class.get_group(103507).index)   
# print(df_by_class.count().shape)
# list(df_by_class)[10]
# train_df.info()

# print(train_df.describe())
predicted=train_sum10.reset_index()

result=pd.merge(test_df[['predict_date','class_id']],predicted,how='left',on=['class_id'])

result.fillna(0)

result.columns=['predict_date','class_id','predict_quantity']

   

result.to_csv('result_201710.csv',index=False,header=True)  

/Users/lvst/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(20157, 32)
140
<class 'dict'>
103507
(129, 32)
124140
(131, 32)
125403
(255, 32)
136916
(346, 32)
169673
(5, 32)
175962
(248, 32)
178529
(298, 32)
186250
(21, 32)
194201
(2, 32)
194450
(168, 32)
198427
(77, 32)
206765
(90, 32)
209945
(304, 32)
219195
(118, 32)
221795
(170, 32)
245609
(58, 32)
248352
(472, 32)
249875
(45, 32)
250658
(30, 32)
265980
(161, 32)
270690
(110, 32)
281301
(328, 32)
281792
(115, 32)
289386
(171, 32)
289403
(60, 32)
290854
(380, 32)
291086
(248, 32)
291514
(147, 32)
302513
(33, 32)
304458
(347, 32)
308913
(219, 32)
321683
(379, 32)
340175
(6, 32)
347384
(375, 32)
348641
(63, 32)
349023
(439, 32)
350259
(131, 32)
354068
(435, 32)
356986
(141, 32)
359378
(2, 32)
360648
(125, 32)
376193
(2, 32)
378356
(45, 32)
378536
(17, 32)
379265
(34, 32)
379876
(189, 32)
384924
(220, 32)
392593
(35, 32)
395234
(143, 32)
401052
(80, 32)
409854
(405, 32)
412240
(33, 32)
416749
(137, 32)
417803
(7, 32)
425432
(480, 32)
436105
(169, 32)
437063
(23, 32)
437598
(88, 32)
453752
(277,

In [3]:
# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.train.images}, y=mnist.train.labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)

In [4]:
# Define the neural network
def neural_net(x_dict):
    # TF Estimator input is a dict, in case of multiple inputs
    x = x_dict['images']
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(x, n_hidden_1)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, n_hidden_2)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, num_classes)
    return out_layer

In [5]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    logits = neural_net(features)
    
    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [6]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x108cf9a10>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/54/zw52xh6x6bg1qc_ysdccg7080000gp/T/tmpesyILw', '_save_summary_steps': 100}


In [7]:
# Train the Model
model.train(input_fn, steps=num_steps)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/54/zw52xh6x6bg1qc_ysdccg7080000gp/T/tmpesyILw/model.ckpt.
INFO:tensorflow:loss = 2.56586, step = 1
INFO:tensorflow:global_step/sec: 122.116
INFO:tensorflow:loss = 0.277921, step = 101 (0.820 sec)
INFO:tensorflow:global_step/sec: 134.587
INFO:tensorflow:loss = 0.339535, step = 201 (0.743 sec)
INFO:tensorflow:global_step/sec: 136.635
INFO:tensorflow:loss = 0.245928, step = 301 (0.733 sec)
INFO:tensorflow:global_step/sec: 130.395
INFO:tensorflow:loss = 0.207377, step = 401 (0.766 sec)
INFO:tensorflow:global_step/sec: 135.797
INFO:tensorflow:loss = 0.425328, step = 501 (0.736 sec)
INFO:tensorflow:global_step/sec: 139.301
INFO:tensorflow:loss = 0.239763, step = 601 (0.718 sec)
INFO:tensorflow:global_step/sec: 141.19
INFO:tensorflow:loss = 0.221997, step = 701 (0.708 sec)
INFO:tensorflow:global_step/sec: 140.746
INFO:tensorflow:loss = 0.308821, step = 801 (0.710 sec)
INFO:tensorflow:global_

In [ ]:
# Evaluate the Model
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.test.images}, y=mnist.test.labels,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(input_fn)

INFO:tensorflow:Starting evaluation at 2017-12-28-02:12:19
INFO:tensorflow:Restoring parameters from /var/folders/54/zw52xh6x6bg1qc_ysdccg7080000gp/T/tmpesyILw/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-12-28-02:12:20
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.9169, global_step = 1000, loss = 0.288614


{'accuracy': 0.91689998, 'global_step': 1000, 'loss': 0.28861371}

In [ ]:
# Predict single images
n_images = 4
# Get images from test set
test_images = mnist.test.images[:n_images]
# Prepare the input data
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': test_images}, shuffle=False)
# Use the model to predict the images class
preds = list(model.predict(input_fn))

# Display
for i in range(n_images):
    plt.imshow(np.reshape(test_images[i], [28, 28]), cmap='gray')
    plt.show()
    print("Model prediction:", preds[i])

INFO:tensorflow:Restoring parameters from /var/folders/54/zw52xh6x6bg1qc_ysdccg7080000gp/T/tmpesyILw/model.ckpt-1000
